In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# OAI dataset
df = pd.read_csv("data/3.preprocessed_OAI.csv")
# GLAD dataset
dfg = pd.read_csv("data/3.preprocessed_Glad.csv")
df.shape
print("OAI shape:", df.shape, "GLAD shape:", dfg.shape)

OAI shape: (4131, 11) GLAD shape: (7603, 11)


##### Renaming columns in df and dfg for consistency

In [3]:
# Renaming columns in df for consistency
df.rename(columns={
    'V00AGE': 'Age',
    'P02SEX': 'Gender',
    'P01BMI': 'BMI',
    'V00CESD6': 'Depression',
    'V00CEMPLOY_employment': 'Employment_status',
    'V00KOOSQOL': 'Baseline_Quality_of_life',
    'V01KOOSQOL': 'Followup_Quality_of_life',
    'knee_pain_baseline': 'Baseline_Pain',
    'knee_pain_follow': 'Followup_Pain',
    'PA_baseline_category': 'Baseline_Physical_Activity',
    'PA_followup_category': 'Followup_Physical_Activity'
}, inplace=True)

dfg.rename(columns={
    'age': 'Age',
    'gender': 'Gender',
    'fysb_BMI': 'BMI',
    'ptb_14618_depression': 'Depression',
    'ptb_3777_employment': 'Employment_status',
    'ptb_koos_qol_score': 'Baseline_Quality_of_life',
    'pt12_koos_qol_score': 'Followup_Quality_of_life',
    'ptb_3764': 'Baseline_Pain',
    'pt12_3764': 'Followup_Pain',
    'ptb_4145': 'Baseline_Physical_Activity',
    'pt12_4145': 'Followup_Physical_Activity'
}, inplace=True)


##### Make PA to categorical variable

In [4]:
# OAI dataset
##### Physical activity
# Ensure the columns are of type 'category'
df['Followup_Physical_Activity'] = df['Followup_Physical_Activity'].astype('category')
df['Baseline_Physical_Activity'] = df['Baseline_Physical_Activity'].astype('category')

# Convert categories to codes if they are ordinal
df['Followup_Physical_Activity'] = df['Followup_Physical_Activity'].cat.codes
df['Baseline_Physical_Activity'] = df['Baseline_Physical_Activity'].cat.codes

df['PA_change'] = df['Followup_Physical_Activity'] - df['Baseline_Physical_Activity']

##### Quality of life
df['QOL_change'] = df['Followup_Quality_of_life'] - df['Baseline_Quality_of_life']

##### Pain
df['pain_change'] = df['Followup_Pain'] - df['Baseline_Pain']

In [5]:
# GLAD Dataset
##### Physical activity
dfg['Baseline_Physical_Activity'] = dfg['Baseline_Physical_Activity'].astype('category')
dfg['Baseline_Physical_Activity'] = dfg['Baseline_Physical_Activity'].cat.codes

dfg['Followup_Physical_Activity'] = dfg['Followup_Physical_Activity'].astype('category')
dfg['Followup_Physical_Activity'] = dfg['Followup_Physical_Activity'].cat.codes

dfg['PA_change'] = dfg['Followup_Physical_Activity'] - dfg['Baseline_Physical_Activity']

##### Quality of life
dfg['QOL_change'] = dfg['Followup_Quality_of_life'] - dfg['Baseline_Quality_of_life']

##### Pain
dfg['pain_change'] = dfg['Followup_Pain'] - dfg['Baseline_Pain']

In [6]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Followup_Quality_of_life,Baseline_Pain,Followup_Pain,PA_change,QOL_change,pain_change
0,78.0,1.0,27.6,1.0,3.0,0,0,50.00,68.75,80.0,50.0,0,18.75,-30.0
1,76.0,2.0,27.4,1.0,3.0,0,2,100.00,62.50,0.0,40.0,2,-37.50,40.0
2,69.0,1.0,29.8,1.0,3.0,0,3,81.25,100.00,10.0,0.0,3,18.75,-10.0
3,68.0,2.0,30.1,2.0,3.0,0,0,75.00,81.25,30.0,30.0,0,6.25,0.0
4,76.0,1.0,32.4,1.0,2.0,0,0,75.00,68.75,30.0,40.0,0,-6.25,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,8,100.00,100.00,0.0,0.0,-1,0.00,0.0
4127,59.0,2.0,30.5,1.0,1.0,9,5,43.75,68.75,70.0,70.0,-4,25.00,0.0
4128,70.0,1.0,33.9,1.0,3.0,9,4,100.00,100.00,0.0,0.0,-5,0.00,0.0
4129,52.0,1.0,30.7,1.0,1.0,9,7,31.25,37.50,50.0,70.0,-2,6.25,20.0


In [7]:
dfg

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Followup_Quality_of_life,Baseline_Pain,Followup_Pain,PA_change,QOL_change,pain_change
0,50,2,22.68,1.0,1.0,9,9,31.25,56.25,65.0,17.0,0,25.00,-48.0
1,45,1,29.07,1.0,1.0,6,9,25.00,43.75,96.0,52.0,3,18.75,-44.0
2,46,2,26.10,1.0,1.0,2,8,68.75,62.50,39.0,25.0,6,-6.25,-14.0
3,53,2,28.03,1.0,1.0,3,3,62.50,87.50,15.0,2.0,0,25.00,-13.0
4,68,2,27.82,1.0,1.0,7,9,87.50,93.75,15.0,1.0,2,6.25,-14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7598,55,2,35.63,1.0,1.0,7,9,50.00,56.25,57.0,9.0,2,6.25,-48.0
7599,61,2,29.34,1.0,1.0,2,5,25.00,68.75,55.0,8.0,3,43.75,-47.0
7600,62,1,25.88,1.0,1.0,2,2,50.00,25.00,26.0,77.0,0,-25.00,51.0
7601,47,2,26.51,1.0,1.0,3,1,43.75,18.75,47.0,92.0,-2,-25.00,45.0


##### Statistical

In [8]:
df.columns

Index(['Age', 'Gender', 'BMI', 'Depression', 'Employment_status',
       'Baseline_Physical_Activity', 'Followup_Physical_Activity',
       'Baseline_Quality_of_life', 'Followup_Quality_of_life', 'Baseline_Pain',
       'Followup_Pain', 'PA_change', 'QOL_change', 'pain_change'],
      dtype='object')

In [9]:
df.Baseline_Physical_Activity.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

In [10]:
def analyze_data(df, continuous_vars, categorical_vars):
    results = {}
    # Analyze continuous variables
    for var in continuous_vars:
        results[var] = {
            'min': df[var].min(),
            'max': df[var].max(),
            'mean': df[var].mean()
        }
    # Analyze categorical variables
    for var in categorical_vars:
        results[var] = df[var].value_counts().to_dict()
    
    return results

# Continuous and Categorical variables

continuous_vars = ['Age', 'BMI', 'Baseline_Quality_of_life', 'Baseline_Pain', 'QOL_change', 'pain_change']
categorical_vars = ['Gender', 'Depression', 'Employment_status', 'Baseline_Physical_Activity', 'PA_change']

results_df = analyze_data(df, continuous_vars, categorical_vars)
results_dfg = analyze_data(dfg, continuous_vars, categorical_vars)

def print_results(title, results):
    print(title)
    for key, value in results.items():
        print(f"{key}: {value}")
        print()  # This adds an empty line after each variable's result


In [11]:
print_results("Analysis for df:", results_df)

Analysis for df:
Age: {'min': 45.0, 'max': 79.0, 'mean': 61.08545146453643}

BMI: {'min': 16.9, 'max': 48.7, 'mean': 28.563519728879204}

Baseline_Quality_of_life: {'min': 0.0, 'max': 100.0, 'mean': 67.79381505688696}

Baseline_Pain: {'min': 0.0, 'max': 100.0, 'mean': 32.946017913338174}

QOL_change: {'min': -81.25, 'max': 93.75, 'mean': 3.334039377067699}

pain_change: {'min': -100.0, 'max': 100.0, 'mean': -2.6506899055918662}

Gender: {2.0: 2395, 1.0: 1736}

Depression: {1.0: 3217, 2.0: 774, 3.0: 88, 4.0: 52}

Employment_status: {1.0: 2589, 3.0: 1344, 2.0: 198}

Baseline_Physical_Activity: {5: 990, 3: 915, 6: 661, 4: 569, 7: 374, 2: 243, 9: 168, 1: 80, 8: 79, 0: 52}

PA_change: {0: 1315, -1: 746, 1: 739, 2: 394, -2: 381, 3: 188, -3: 173, -4: 74, 4: 59, 5: 20, -5: 20, 6: 9, -6: 8, 7: 3, -7: 2}



In [12]:
print_results("Analysis for dfg:", results_dfg)

Analysis for dfg:
Age: {'min': 23, 'max': 85, 'mean': 56.69222675259766}

BMI: {'min': 17.03, 'max': 72.27, 'mean': 29.592791003551234}

Baseline_Quality_of_life: {'min': 0.0, 'max': 100.0, 'mean': 45.055405760883865}

Baseline_Pain: {'min': 0.0, 'max': 100.0, 'mean': 47.21938708404577}

QOL_change: {'min': -50.0, 'max': 81.25, 'mean': 9.630518654916917}

pain_change: {'min': -96.0, 'max': 88.0, 'mean': -14.541628304616598}

Gender: {2: 5324, 1: 2279}

Depression: {1.0: 7188, 2.0: 314, 3.0: 88, 4.0: 13}

Employment_status: {1.0: 6782, 2.0: 451, 3.0: 370}

Baseline_Physical_Activity: {5: 1694, 3: 1484, 6: 1237, 4: 1162, 7: 628, 2: 481, 9: 422, 8: 267, 1: 194, 0: 34}

PA_change: {0: 2359, 1: 1245, -1: 1017, 2: 959, -2: 606, 3: 465, 4: 270, -3: 244, -4: 140, 5: 110, -5: 53, 6: 45, -6: 35, 7: 23, 8: 12, -7: 11, -8: 6, 9: 2, -9: 1}



##### Physical activity

In [13]:
dfP = df.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "QOL_change", "pain_change"])
dfgP = dfg.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "QOL_change", "pain_change"])

In [14]:
dfP 

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change
0,78.0,1.0,27.6,1.0,3.0,0,50.00,80.0,0
1,76.0,2.0,27.4,1.0,3.0,0,100.00,0.0,2
2,69.0,1.0,29.8,1.0,3.0,0,81.25,10.0,3
3,68.0,2.0,30.1,2.0,3.0,0,75.00,30.0,0
4,76.0,1.0,32.4,1.0,2.0,0,75.00,30.0,0
...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,100.00,0.0,-1
4127,59.0,2.0,30.5,1.0,1.0,9,43.75,70.0,-4
4128,70.0,1.0,33.9,1.0,3.0,9,100.00,0.0,-5
4129,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,-2


In [15]:
dfgP

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,PA_change
0,50,2,22.68,1.0,1.0,9,31.25,65.0,0
1,45,1,29.07,1.0,1.0,6,25.00,96.0,3
2,46,2,26.10,1.0,1.0,2,68.75,39.0,6
3,53,2,28.03,1.0,1.0,3,62.50,15.0,0
4,68,2,27.82,1.0,1.0,7,87.50,15.0,2
...,...,...,...,...,...,...,...,...,...
7598,55,2,35.63,1.0,1.0,7,50.00,57.0,2
7599,61,2,29.34,1.0,1.0,2,25.00,55.0,3
7600,62,1,25.88,1.0,1.0,2,50.00,26.0,0
7601,47,2,26.51,1.0,1.0,3,43.75,47.0,-2


In [16]:
dfP.to_csv('data/4.PA_control.csv', index=False)
dfgP.to_csv('data/4.PA_treatment.csv', index=False)

##### Quality of life

In [17]:
dfQ = df.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "PA_change", "pain_change"])
dfgQ = dfg.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "PA_change", "pain_change"])

In [18]:
dfQ.to_csv('data/4.QOL_control.csv', index=False)
dfgQ.to_csv('data/4.QOL_treatment.csv', index=False)

##### Pain

In [19]:
dfpp = df.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "PA_change", "QOL_change"])
dfgp = dfg.drop(columns=["Followup_Physical_Activity", "Followup_Quality_of_life", "Followup_Pain", "PA_change", "QOL_change"])

In [20]:
dfpp.to_csv('data/4.Pain_control.csv', index=False)
dfgp.to_csv('data/4.Pain_treatment.csv', index=False)

In [21]:
dfpp

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Quality_of_life,Baseline_Pain,pain_change
0,78.0,1.0,27.6,1.0,3.0,0,50.00,80.0,-30.0
1,76.0,2.0,27.4,1.0,3.0,0,100.00,0.0,40.0
2,69.0,1.0,29.8,1.0,3.0,0,81.25,10.0,-10.0
3,68.0,2.0,30.1,2.0,3.0,0,75.00,30.0,0.0
4,76.0,1.0,32.4,1.0,2.0,0,75.00,30.0,10.0
...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,9,100.00,0.0,0.0
4127,59.0,2.0,30.5,1.0,1.0,9,43.75,70.0,0.0
4128,70.0,1.0,33.9,1.0,3.0,9,100.00,0.0,0.0
4129,52.0,1.0,30.7,1.0,1.0,9,31.25,50.0,20.0
